# Storage Management

- List, Create, and, Modify Storag
  - `lsblk` can be used to list partitions
    ```
    [azureadmin@centos01 /]$ lsblk
    NAME    MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
    sda       8:0    0   30G  0 disk
    ├─sda1    8:1    0  500M  0 part /boot
    ├─sda2    8:2    0   29G  0 part /
    ├─sda14   8:14   0    4M  0 part
    └─sda15   8:15   0  495M  0 part /boot/efi
    sdb       8:16   0   64G  0 disk
    └─sdb1    8:17   0   64G  0 part /mnt/resource
    ```
  - All block devices can be found in the `/dev/` directory
  - `sudo fdisk -l /dev/sda` can be used to view detailed info about partitions on a block device
  - `cfdisk` can be used to creat a partition on a disk, and is easier to use that `fdisk` 
    - `cfdisk` can be used to resize existing partitions

- Create and Manage Swap Space
  - swap is used when no RAM is available
  - swapping is also known as paging
  - swapping is almost always slower than working with RAM
  - `mkswap` can be used to create swap space
    - `sudo mkwap /dev/sda2`
  - `swapon` can be used to work with swap space
    - `swapon --show`
  - `dd` can be used to write random data to a disk
    - `sudo dd if=/dev/zero of=/dev/sdc2 bs=1M count=128 status=progress`

- Create and Configure File Systems
  - To create a XFS file system:
    - `sudo mkfs.xfs /dev/sdb1`
  - To create a EXT4 file system:
    - `sudo mkfs.ext4 /dev/sdb1`
  - `xfs_admin` can be used to change things on an already created xfs file system
  - `tune2fs`

- Configure Systems to Mount File Systems at Boot Time
  - File systems are typically mounted in `/mnt`
  - To mount a file system:
    - `sudo mount /dev/vdb1 /mnt/vdb1`
  - To unmount a file system:
    - `sudo umount /mnt/vdb1`
  - `/etc/fstab` contains persistent mount information for disks
  - `swaplabel` can be used to configure the label of swap partitions
    - `sudo swaplabel -L myLabel /dev/sda1`
